In [1]:
import extrack
import numpy as np
from matplotlib import pyplot as plt
import multiprocessing

version: 1.4


In this tutorial, we will have an overview of all the possible kind of data sets that can be used in the fitting module of ExTrack and how to instanciate parameters depending of the data set.





In [2]:
dt = 0.02

all_tracks, all_Bs, all_sigmas = extrack.simulate_tracks.sim_FOV(nb_tracks=10000,
                                                     max_track_len=5,
                                                     LocErr=np.array([0.02, 0.02, 0.04]), # localization error in x, y and z (even if not used)
                                                     Ds = np.array([0,0.25]),
                                                     initial_fractions = np.array([0.6,0.4]),
                                                     TrMat = np.array([[0.9,0.1],[0.1,0.9]]),
                                                     dt = dt,
                                                     nb_dims = 2,
                                                     LocErr_std = 1e-10, # normalized standard deviation of localization error (coef of variation)
                                                     pBL = 0.001,
                                                     cell_dims = [100,None,None], # dimension limits in x, y and z respectively
                                                     min_len = 5)


number of tracks:
5 pos : 9897, 


In [4]:
# fit parameters of the simulated tracks : 
# increase the number of workers accordingly to your number of avaialable core for faster computing, not working on windows

#First, parameters of the model have to be set, parameters of a 2-state can be initialized as such:
nb_states = 2
steady_state = False

vary_params = {'LocErr' : True, 'D0' : True, 'D1' : True, 'F0' : True, 'p01' : True, 'p10' : True, 'pBL' : False}
estimated_vals =  {'LocErr' : 0.025, 'D0' : 1e-20, 'D1' : 0.05, 'F0' : 0.45, 'p01' : 0.05, 'p10' : 0.05, 'pBL' : 0.1}
min_values = {'LocErr' : 0.007, 'D0' : 1e-12, 'D1' : 0.00001, 'F0' : 0.001, 'p01' : 0.001, 'p10' : 0.001, 'pBL' : 0.001}
max_values = {'LocErr' : 0.6, 'D0' : 1, 'D1' : 10, 'F0' : 0.999, 'p01' : 1., 'p10' : 1., 'pBL' : 0.99}

# In case of more states, on can use the same technique by adding the correct elements to the vary_params, estimated_vals, min_values and max_values dictionaries

params =  extrack.tracking.get_params(nb_states = nb_states, 
                    steady_state = steady_state,
                    vary_params = vary_params,
                    estimated_vals = estimated_vals,
                    min_values = min_values,
                    max_values = max_values)

cell_dims = [100]
nb_substeps = 1
nb_states = 2
frame_len = 5
verbose = 0
workers = 1
input_LocErr = None
method = 'powell'
steady_state = False

model_fit = extrack.tracking.param_fitting(all_tracks,
                                           params = params,
                                           dt = dt,
                                           cell_dims = [100],
                                           nb_substeps = 1,
                                           nb_states = 2,
                                           frame_len = 5,
                                           verbose = 0,
                                           workers = 1,
                                           input_LocErr = None,
                                           method = 'powell',
                                           steady_state = False)

for param in model_fit.params:
    print(param + ':', np.round(model_fit.params[param].value, 6), ',', end = '')


.........................................................................................................................................................................................................................................................................................................................................................................................................


## Using the function generate params

The function generate_params allows to create different format of parameters depending on the user needs.

### Function extrack.tracking.generate_params

arguments:

- nb_states: number of states of the model.
- LocErr_type: Type of localization errors which can be specified, unique/multiple localization error parameters, input localization error for each peak based on peaks, etc. See below for more information. 
- nb_dims: number of spatial dimensions, only matters if LocErr_type == 2, which stands for independent localization error parameters for each dimension.
- LocErr_bounds: list of minimal and maximal values allowed for localization error parameters. e.g. [0.005, 0.1]., The initial guess on LocErr will be the geometric mean of the boundaries if estimated_LocErr = None.
- D_max: maximum diffusion coefficient allowed.
- Fractions_bounds: list of minimum and maximum values for initial fractions.
- estimated_LocErr: One can specify a list of initial guesses for localization error for each dimension. The list must contain 1 element if LocErr_type = 1, nb_dims elements if LocErr_type = 2 (for each dimension) or 2 elements if LocErr_type = 3 (the first element for x,y axes and the second for z axis).
- estimated_Ds: list of estimated D (nb_states elements), D will be arbitrary interspaced from 0 to D_max if None, otherwise input 1D array/list of Ds for each state from state 0 to nb_states - 1.
- estimated_Fs: list of estimated initial fractions (nb_states elements). Fractions will be initialized as equal if None (1/nb_states), otherwise input 1D array/list of fractions for each state from state 0 to nb_states - 1.
- estimated_transition_rates: initial guess of the values for transition rate per step. can be a float (from 0 to 1), in this case all rates will be initialized with the same float value. Can also be a 1D array or list of transition rates from state i to j varying the i then the j, for instance [k01, k02, k10, k12, k20, k21] in case of a 3-state model. The number of elements must be nb_states * (nb_states -1).
- slope_offsets_estimates: If LocErr_type = 4, list of Initial guesses for the slope and offset where the peak-wise proxi for localization error input_LocErr is linked to the actual localization error such that: localization error = slope * input_LocErr + offset.
)

Output:

- params: list of parameters to fit.

## The LocErr_type argument:

The LocErr_type argument allows to create a list of parameters that is most suited to the experiments: If None Localization error will not be considered as a parameter and thus won't be fitted. In this specific case, the user will have to use input_LocErr in the fitting function to inform the model about localization error for each peak (See below). 

- LocErr_type = 1: For a single localization error parameter, all spatial dimensions will use the same localization error.
- LocErr_type = 2: For a localization error parameter for each dimension. Localization error parameter for dim i will be 'LocErr'+str(i).
- LocErr_type = 3: For a shared localization error for x and y dims (the 2 first dimensions) and another for z dimension. The z dimension has to be the 3rd in the input tracks.
- LocErr_type = 4: Can be used when the user has a measurement of the peak quality which is not exactly the localization error. It will draw an affine relationship between localization error according to ExTrack and the peak-wise input specified with input_LocErr (an estimate of localization error/quality of peak/signal to noise ratio, etc).
- LocErr_type = None: For no localization error fits, localization error is then directly assumed from a prior peak-wise estimate of localization error specified in input_LocErr.

See below for practical examples using the different LocErr_type values:

In [6]:
# as mentioned above, LocErr_type = 1 creates a single localization error parameter for all peaks:

params = extrack.tracking.generate_params(nb_states = 2,
                         LocErr_type = 1, # Type of localization error to be used, see https://github.com/vanTeeffelenLab/ExTrack/wiki/Localization-Error for more informations,
                         LocErr_bounds = [0.005, 0.1], # the initial guess on LocErr will be the geometric mean of the boundaries
                         D_max = 3, # maximal diffusion length allowed
                         Fractions_bounds = [0.001, 0.99],
                         estimated_transition_rates = 0.1 # transition rate per step.
                         )

model_fit = extrack.tracking.param_fitting(all_tracks = all_tracks,
                                           dt = dt,
                                           params = params,
                                           cell_dims = [100],
                                           nb_substeps = 1,
                                           nb_states = 2,
                                           frame_len = 5,
                                           verbose = 0,
                                           workers = 1,
                                           input_LocErr = None,
                                           method = 'powell',
                                           steady_state = False)

for param in model_fit.params:
    print(param + ':', np.round(model_fit.params[param].value, 6), ',', end = '')
    

...x...............................................................................................xx.x.....................................................................................
Parameters([('D0', <Parameter 'D0', value=0.0003069071470827578, bounds=[0:3]>), ('D1', <Parameter 'D1', value=0.25344110240304135, bounds=[0:3]>), ('LocErr', <Parameter 'LocErr', value=0.019843255432158303, bounds=[0.005:0.1]>), ('F0', <Parameter 'F0', value=0.6101541326726733, bounds=[0.001:0.99]>), ('F1', <Parameter 'F1', value=0.38984586732732673, bounds=[-inf:inf], expr='1 - F0'>), ('p01', <Parameter 'p01', value=0.09741551432756747, bounds=[0.0001:1]>), ('p10', <Parameter 'p10', value=0.09341863899436702, bounds=[0.0001:1]>), ('pBL', <Parameter 'pBL', value=0.05657639536566579, bounds=[0.0001:1]>)])


In [11]:
# Estimates of each parameter can also be specified as followed by using the arguments
# estimated_LocErr, estimated_Ds, estimated_Fs and estimated_transition_rates:

params = extrack.tracking.generate_params(nb_states = 2,
                    LocErr_type = 1,
                    nb_dims = 3, # only matters if LocErr_type == 2,
                    LocErr_bounds = [0.005, 0.1], # the initial guess on LocErr will be the geometric mean of the boundaries
                    D_max = 10, # maximal diffusion coefficient allowed
                    Fractions_bounds = [0.001, 0.99],
                    estimated_LocErr = [0.02],
                    estimated_Ds = [0, 0.25], # input a list of Ds for each state from state 0 to nb_states - 1. If None, Ds will be spaced from 0 to D_max
                    estimated_Fs = [0.5, 0.5], # one can input a list of fractions for each state from state 0 to nb_states - 1. If None all fractions will be equal.
                    estimated_transition_rates = [0.1, 0.1], # transition rate per step. [0.1,0.05,0.03,0.07,0.2,0.2]
                    slope_offsets_estimates = None # need to specify the list [slop, offset] if LocErr_type = 4,
                    )

model_fit = extrack.tracking.param_fitting(all_tracks = all_tracks,
                                           dt = dt,
                                           params = params,
                                           cell_dims = [100],
                                           nb_substeps = 1,
                                           nb_states = 2,
                                           frame_len = 5,
                                           verbose = 0,
                                           workers = 1,
                                           input_LocErr = None,
                                           method = 'powell',
                                           steady_state = False)

for param in model_fit.params:
    print(param + ':', np.round(model_fit.params[param].value, 6), ',', end = '')
    

..xx.xx...................................................................................................xx.xx.....................................................................................
Parameters([('D0', <Parameter 'D0', value=0.00013127625959619138, bounds=[0:10]>), ('D1', <Parameter 'D1', value=0.25249584050548834, bounds=[0:10]>), ('LocErr', <Parameter 'LocErr', value=0.019985955236071697, bounds=[0.005:0.1]>), ('F0', <Parameter 'F0', value=0.6099362533126705, bounds=[0.001:0.99]>), ('F1', <Parameter 'F1', value=0.39006374668732946, bounds=[-inf:inf], expr='1 - F0'>), ('p01', <Parameter 'p01', value=0.09788417685783249, bounds=[0.0001:1]>), ('p10', <Parameter 'p10', value=0.09387844733673723, bounds=[0.0001:1]>), ('pBL', <Parameter 'pBL', value=0.05298821956893871, bounds=[0.0001:1]>)])


In [15]:
# LocErr_type = 2 allows to use one localization error parameters for each spacial dimension :
# in this case for instance we generate 3D tracks with localization errors of 0.018 um in x, 0.022 um in y and 0.04 um in z.

all_tracks, all_Bs, all_sigs = extrack.simulate_tracks.sim_FOV(nb_tracks=10000,
                                                     max_track_len=5,
                                                     LocErr=np.array([0.018, 0.022, 0.04]),
                                                     Ds = np.array([0,0.25]),
                                                     initial_fractions = np.array([0.6,0.4]),
                                                     TrMat = np.array([[0.9,0.1],[0.1,0.9]]),
                                                     dt = dt,
                                                     nb_dims = 3,
                                                     LocErr_std = 1e-10, # normalized standard deviation of localization error (coef of variation)
                                                     pBL = 0.001,
                                                     cell_dims = [100,None,None], # dimension limits in x, y and z respectively
                                                     min_len = 5)
    
params = extrack.tracking.generate_params(nb_states = 2,
                         LocErr_type = 2, # Type of localization error to be used, see https://github.com/vanTeeffelenLab/ExTrack/wiki/Localization-Error for more informations,
                         nb_dims = 3, # number of spatial dimensions of the tracks, only matter if LocErr_type == 2,
                         LocErr_bounds = [0.005, 0.1], # the initial guess on LocErr will be the geometric mean of the boundaries
                         D_max = 1, # maximal diffusion length allowed
                         Fractions_bounds = [0.001, 0.99],
                         estimated_transition_rates = 0.1 # transition rate per step.
                         )

model_fit = extrack.tracking.param_fitting(all_tracks,
                                           params = params,
                                           dt = dt,
                                           cell_dims = [100],
                                           nb_substeps = 1,
                                           nb_states = 2,
                                           frame_len = 5,
                                           verbose = 0,
                                           workers = 1,
                                           input_LocErr = None,
                                           method = 'powell',
                                           steady_state = False)

for param in model_fit.params:
    print(param + ':', np.round(model_fit.params[param].value, 6), ',    ', end = '')


number of tracks:
5 pos : 9915, 
...x...........x.........................................................................................xx..........................................................................................................................................................................................................................................................................................................................................................................................................................................................
D0: 0.000172 ,    D1: 0.257219 ,    LocErr0: 0.017988 ,    LocErr1: 0.021815 ,    LocErr2: 0.039941 ,    F0: 0.609309 ,    F1: 0.390691 ,    p01: 0.096013 ,    p10: 0.104264 ,    pBL: 0.281303 ,    

## LocErr_type = 3

LocErr_type = 3 allows to use a unique localization error parameters for x and y axes and another for z axis:


In [17]:

all_tracks, all_Bs, all_sigs = extrack.simulate_tracks.sim_FOV(nb_tracks=10000,
                                                     max_track_len=5,
                                                     LocErr=np.array([0.02, 0.02, 0.04]),
                                                     Ds = np.array([0,0.25]),
                                                     initial_fractions = np.array([0.6,0.4]),
                                                     TrMat = np.array([[0.9,0.1],[0.1,0.9]]),
                                                     dt = dt,
                                                     nb_dims = 3,
                                                     LocErr_std = 1e-10, # normalized standard deviation of localization error (coef of variation)
                                                     pBL = 0.001,
                                                     cell_dims = [100,None,None], # dimension limits in x, y and z respectively
                                                     min_len = 5)
    
params = extrack.tracking.generate_params(nb_states = 2,
                         LocErr_type = 3, # Type of localization error to be used, see https://github.com/vanTeeffelenLab/ExTrack/wiki/Localization-Error for more informations,
                         nb_dims = 3, # number of spatial dimensions of the tracks, only matter if LocErr_type == 2,
                         LocErr_bounds = [0.005, 0.1], # the initial guess on LocErr will be the geometric mean of the boundaries
                         D_max = 1, # maximal diffusion length allowed
                         Fractions_bounds = [0.001, 0.99],
                         estimated_transition_rates = 0.1 # transition rate per step.
                         )

model_fit = extrack.tracking.param_fitting(all_tracks,
                                           params = params,
                                           dt = dt,
                                           cell_dims = [100],
                                           nb_substeps = 1,
                                           nb_states = 2,
                                           frame_len = 5,
                                           verbose = 0,
                                           workers = 1,
                                           input_LocErr = None,
                                           method = 'powell',
                                           steady_state = False)

for param in model_fit.params:
    print(param + ':', np.round(model_fit.params[param].value, 6), ',    ', end = '')


number of tracks:
5 pos : 9910, 
...x...........x..............................................................................xx......................................................................................................................................................................................................................................................................................................................................................................................................
D0: 0.00015 ,    D1: 0.256806 ,    LocErr0: 0.01997 ,    LocErr1: 0.01997 ,    LocErr2: 0.039961 ,    F0: 0.614916 ,    F1: 0.385084 ,    p01: 0.099082 ,    p10: 0.103153 ,    pBL: 0.068394 ,    

## LocErr_type = None

LocErr_type = None allows to use an estimate of localization error as input for each peak specified by the argument input_LocErr).

input_LocErr must have the type of structure than all_tracks (dict of array with track lengths as keys and arrays of dims (number of tracks, track length, number of spatial dimensions).

values of input_LocErr has to be computed from the movies prior tracking, the reader functions allow to collect these values using the argument opt_metrics_names for read_trackmate_xml and opt_colnames for read_table.


In [4]:

all_tracks, all_Bs, all_sigs = extrack.simulate_tracks.sim_FOV(nb_tracks=10000,
                                                     max_track_len=5,
                                                     LocErr=np.array([0.018, 0.022, 0.04]),
                                                     Ds = np.array([0,0.25]),
                                                     initial_fractions = np.array([0.6,0.4]),
                                                     TrMat = np.array([[0.9,0.1],[0.1,0.9]]),
                                                     dt = dt,
                                                     nb_dims = 3,
                                                     LocErr_std = 0.2, # normalized standard deviation of localization error (coef of variation)
                                                     pBL = 0.001,
                                                     cell_dims = [100,None,None], # dimension limits in x, y and z respectively
                                                     min_len = 5)

params = extrack.tracking.generate_params(nb_states = 2,
                         LocErr_type = None, # Type of localization error to be used, see https://github.com/vanTeeffelenLab/ExTrack/wiki/Localization-Error for more informations,
                         nb_dims = 3, # number of spatial dimensions of the tracks, only matter if LocErr_type == 2,
                         LocErr_bounds = [0.005, 0.1], # the initial guess on LocErr will be the geometric mean of the boundaries
                         D_max = 1, # maximal diffusion length allowed
                         Fractions_bounds = [0.001, 0.99],
                         estimated_transition_rates = 0.1 # transition rate per step.
                         )

model_fit = extrack.tracking.param_fitting(all_tracks,
                                           params = params,
                                           dt = dt,
                                           cell_dims = [100],
                                           nb_substeps = 1,
                                           nb_states = 2,
                                           frame_len = 5,
                                           verbose = 0,
                                           workers = 1,
                                           input_LocErr = all_sigs,
                                           method = 'powell',
                                           steady_state = False)

for param in model_fit.params:
    print(param + ':', np.round(model_fit.params[param].value, 6), ',    ', end = '')


number of tracks:
5 pos : 9913, 
...x.....................................................................x.......................................................................................................................................................
D0: 8.7e-05 ,    D1: 0.254977 ,    F0: 0.60567 ,    F1: 0.39433 ,    p01: 0.096369 ,    p10: 0.102365 ,    pBL: 0.771589 ,    

In [3]:
path = './tracks.xml'

all_tracks, all_frames, opt_metrics = extrack.readers.read_trackmate_xml(path,
                                       lengths=np.arange(5,50),
                                       dist_th = 0.3,
                                       frames_boundaries = [-np.inf, np.inf],
                                       remove_no_disp = True, 
                                       opt_metrics_names = ['LocErrs0', 'LocErrs1'], # e.g. ['pred_0', 'pred_1'],
                                       opt_metrics_types = None # will assume 'float64' type if none, otherwise specify a list of same length as opt_metrics_names: e.g. ['float64','float64']
                                       )

params = extrack.tracking.generate_params(nb_states = 2,
                         LocErr_type = None, # Type of localization error to be used, see https://github.com/vanTeeffelenLab/ExTrack/wiki/Localization-Error for more informations,
                         nb_dims = 3, # number of spatial dimensions of the tracks, only matter if LocErr_type == 2,
                         LocErr_bounds = [0.005, 0.1], # the initial guess on LocErr will be the geometric mean of the boundaries
                         D_max = 1, # maximal diffusion length allowed
                         Fractions_bounds = [0.001, 0.99],
                         estimated_transition_rates = 0.1 # transition rate per step.
                         )

# Here, opt_metrics is a dict of dicts of keys opt_metrics_names so 'LocErrs0' and 'LocErrs1'
# opt_metrics['LocErrs0'] and opt_metrics['LocErrs1'] are then 2D arrays, for each time position (dim 1) of every track (dim 0).
# We need to concatenate locErrs0 and locErrs1 to fit the format of all_tracks (dim 0: track ID, dim 1: time position, dim 2: spatial dims):
input_LocErr = {}
for l in all_tracks.keys():
    input_LocErr[l] = np.concatenate((opt_metrics['LocErrs0'][l][:,:,None], opt_metrics['LocErrs1'][l][:,:,None]),axis = 2) # concatenate on the axis 2 which represents the spatial dims
    
model_fit = extrack.tracking.param_fitting(all_tracks,
                                           params = params,
                                           dt = dt,
                                           cell_dims = [100],
                                           nb_substeps = 1,
                                           nb_states = 2,
                                           frame_len = 5,
                                           verbose = 0,
                                           workers = 1,
                                           input_LocErr = input_LocErr,
                                           method = 'powell',
                                           steady_state = False)

for param in model_fit.params:
    print(param + ':', np.round(model_fit.params[param].value, 6), ',    ', end = '')
    

...x.....................................................................x...........................................................................................................................................................
D0: 4.9e-05 ,    D1: 0.230044 ,    F0: 0.496684 ,    F1: 0.503316 ,    p01: 0.086127 ,    p10: 0.114738 ,    pBL: 0.128995 ,    

In [4]:
path = './tracks.csv'
# headers of the csv above are: X,Y,frame,track_ID,pred_0,pred_1,LocErrs0,LocErrs1
# We then need to change the argument colnames accordingly.

all_tracks, frames, opt_metrics = extrack.readers.read_table(path, # path of the file to read or list of paths
                                      lengths = np.arange(5,16), # number of positions per track accepted (take the first position if longer than max
                                      dist_th = 0.3, # maximum distance allowed for consecutive positions 
                                      frames_boundaries = [0, 10000], # 
                                      fmt = 'csv', # format of the document to be red, 'csv' or 'pkl', one can also just specify a separator e.g. ' '. 
                                      colnames = ['X', 'Y', 'frame', 'track_ID'], 
                                      opt_colnames = ['LocErrs0', 'LocErrs1'], # list of additional metrics to collect e.g. ['QUALITY', 'TRACK_ID']
                                      remove_no_disp = True)

# Here, opt_metrics is a dict of dicts of keys opt_metrics_names so 'LocErrs0' and 'LocErrs1'
# opt_metrics['LocErrs0'] and opt_metrics['LocErrs1'] are then 2D arrays, for each time position (dim 1) of every track (dim 0).
# We need to concatenate locErrs0 and locErrs1 to fit the format of all_tracks (dim 0: track ID, dim 1: time position, dim 2: spatial dims):

params = extrack.tracking.generate_params(nb_states = 2,
                         LocErr_type = None, # Type of localization error to be used, see https://github.com/vanTeeffelenLab/ExTrack/wiki/Localization-Error for more informations,
                         nb_dims = 3, # number of spatial dimensions of the tracks, only matter if LocErr_type == 2,
                         LocErr_bounds = [0.005, 0.1], # the initial guess on LocErr will be the geometric mean of the boundaries
                         D_max = 1, # maximal diffusion length allowed
                         Fractions_bounds = [0.001, 0.99],
                         estimated_transition_rates = 0.1 # transition rate per step.
                         )

input_LocErr = {}
for l in all_tracks.keys():
    input_LocErr[l] = np.concatenate((opt_metrics['LocErrs0'][l][:,:,None], opt_metrics['LocErrs1'][l][:,:,None]),axis = 2) # concatenate on the axis 2 which represents the spatial dims
    
model_fit = extrack.tracking.param_fitting(all_tracks,
                                           params = params,
                                           dt = dt,
                                           cell_dims = [100],
                                           nb_substeps = 1,
                                           nb_states = 2,
                                           frame_len = 5,
                                           verbose = 0,
                                           workers = 1,
                                           input_LocErr = input_LocErr,
                                           method = 'powell',
                                           steady_state = False)

for param in model_fit.params:
    print(param + ':', np.round(model_fit.params[param].value, 6), ',    ', end = '')


...x...........................................................................x.........................................................................
D0: 1.1e-05 ,    D1: 0.476311 ,    F0: 0.441395 ,    F1: 0.558605 ,    p01: 0.090904 ,    p10: 0.109544 ,    pBL: 0.144649 ,    

## LocErr_type = 4

LocErr_type = 4 allows to use the same kind of estimates than with LocErr_type = None (see above)  and perform a linear fit (y = a*x + b) to find the best estimate of localization error:

input_LocErr can be the Cramer-Rao Lower bound estimate of localization error, the signal to noise ratio, a quality estimator, etc.

In [7]:

# LocErr_type = 4 allows to use the same kind of estimates than with LocErr_type = None (see above)  and perform a linear fit (y = a*x + b) to find the best estimate of localization error:
# input_LocErr can be the Cramer-Rao Lower bound estimate of localization error, the signal to noise ratio, a quality estimator, etc.
all_tracks, all_Bs, all_sigs = extrack.simulate_tracks.sim_FOV(nb_tracks=10000,
                                                     max_track_len=5,
                                                     LocErr=np.array([0.018, 0.022, 0.04]),
                                                     Ds = np.array([0,0.25]),
                                                     initial_fractions = np.array([0.6,0.4]),
                                                     TrMat = np.array([[0.9,0.1],[0.1,0.9]]),
                                                     dt = dt,
                                                     nb_dims = 3,
                                                     LocErr_std = 0.2, # normalized standard deviation of localization error (coef of variation)
                                                     pBL = 0.001,
                                                     cell_dims = [100,None,None], # dimension limits in x, y and z respectively
                                                     min_len = 5)
#linear regression of type: LocErr = Metric * slope + offset  <==>  Metric = (LocErr - offset) / slope

slop = 2
offset = -0.01
input_LocErr = {}
for l in all_sigs.keys():
    input_LocErr[l] = (all_sigs[l] - offset)  / slop 

params = extrack.tracking.generate_params(nb_states = 2,
                         LocErr_type = 4, # Type of localization error to be used, see https://github.com/vanTeeffelenLab/ExTrack/wiki/Localization-Error for more informations,
                         nb_dims = 3, # number of spatial dimensions of the tracks, only matter if LocErr_type == 2,
                         LocErr_bounds = [0.005, 0.1], # the initial guess on LocErr will be the geometric mean of the boundaries
                         D_max = 1, # maximal diffusion length allowed
                         Fractions_bounds = [0.001, 0.99],
                         estimated_transition_rates = 0.1, # transition rate per step.
                         slope_offsets_estimates = [1.8, -0.05]
                         )

model_fit = extrack.tracking.param_fitting(all_tracks,
                                           params = params,
                                           dt = dt,
                                           cell_dims = [100],
                                           nb_substeps = 1,
                                           nb_states = 2,
                                           frame_len = 5,
                                           verbose = 0,
                                           workers = 1,
                                           input_LocErr = input_LocErr,
                                           method = 'powell',
                                           steady_state = False)

# We can then also obtain an estimate of the actual localization error:
Estimated_LocErrs = {}
for l in input_LocErr.keys():
    Estimated_LocErrs[l] = model_fit.params['slope_LocErr'].value * input_LocErr[l] + model_fit.params['offset_LocErr'].value 

for param in model_fit.params:
    print(param + ':', np.round(model_fit.params[param].value, 6), ',    ', end = '')
    

number of tracks:
5 pos : 9934, 
...x.x..........x...............................................................................................x.xx..................................................................................................................x................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................